## Packages and Configuration


In [180]:
# %pip install plotly.express
# %pip install geopandas
# %pip install nbformat
# %pip install pydantic_extra_types

In [181]:
# Suppress hpy5 dependency warning from tslearn
import warnings
warnings.filterwarnings("ignore")

In [182]:
# data libaries
from tslearn.metrics import dtw
from datetime import datetime, timedelta
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math, json
import heapq

# import plotly.express as px
# import geopandas as gpd

In [183]:
# Models
from models.ride import Ride

## Database Configuration

In [184]:
from pymongo import MongoClient
import redis.asyncio as redis
from dotenv import load_dotenv
import certifi, os, uuid

In [185]:
load_dotenv("../.env.prod")

True

In [186]:
# get from environment
db_str = os.getenv("DB_STRING")
client = MongoClient(db_str, tlsCAFile=certifi.where(), uuidRepresentation='standard', tz_aware=True)
database = client.get_database("db")

In [187]:
# get from environment
host = os.getenv("REDIS_HOST")
port = os.getenv("REDIS_PORT")
username = os.getenv("REDIS_USER")
password = os.getenv("REDIS_PASS")

# establish connection
rdb = redis.Redis(host=host, port=port, username=username, password=password)

## RideNow Matching Approach

1. Temporal, limit our candidate set to passengers traveling the same time
2. Spatial, further limit our candidate to passengers with proximity to driver trajectory
3. Correlation, find the driver with highest correlation trajectory in the candidate subset

### Load Trajectory Data


In [188]:
def geojson_to_df(data) -> pd.DataFrame:
    # Extract passenger data
    coordinates = data["route"]["shape"]["coordinates"]
    # timestamps = [datetime.isoformat(ts) for ts in data["route"]["timestamps"]]

    # Create DataFrame
    df = pd.DataFrame({
        "lat": [coord[1] for coord in coordinates],
        "lon": [coord[0] for coord in coordinates], 
        # "timestamp": pd.to_datetime(timestamps)
    })

    return df

In [189]:
with open("./search_p.json", "r") as file:
    data_passenger = json.load(file)

In [190]:
with open("./search_d.json", "r") as file:
    data_driver = json.load(file)

### Trajectory Visualiazation


In [191]:
df_d = geojson_to_df(data_driver)
df_p = geojson_to_df(data_passenger)

In [192]:
df_p.head()

,lat,lon
0,55.65072,12.54122
1,55.65001,12.54043
2,55.64473,12.54384
3,55.64289,12.54686
4,55.64053,12.55380


In [193]:
df_d.head()

,lat,lon
0,55.65072,12.54122
1,55.65001,12.54043
2,55.64473,12.54384
3,55.64289,12.54686
4,55.64053,12.55380


In [194]:
fig = go.Figure()

# add passenger and driver trajectories as traces
fig.add_trace(go.Scattermapbox(mode="lines", lon=df_p["lon"], lat=df_p["lat"], line_color="blue"))
fig.add_trace(go.Scattermapbox(mode="lines", lon=df_d["lon"], lat=df_d["lat"], line_color="purple"))

fig.update_layout(height=600, mapbox=dict(style="open-street-map", zoom=6, center=dict(lon=df_p["lon"].mean(), lat=df_p["lat"].mean())))
fig.update_layout(dragmode=False, uirevision="lock")

### Data conversion


In [195]:
def convert_to_unix(timestamp) -> float:
    datetime_obj = datetime.fromisoformat(timestamp.replace("Z", "+00:00"))
    unix_time = datetime_obj.timestamp()
    return unix_time

In [196]:
def convert_data(matrix):
    # Convert the timestamps to Unix time using pandas (you could also use datetime module in Python)
    timestamps = [convert_to_unix(date) for _, _, date in matrix]

    # Create a numpy array for latitudes, longitudes, and Unix timestamps
    latitudes = np.array([lat for lat, _, _ in matrix])
    longitudes = np.array([lng for _, lng, _ in matrix])
    timestamps = np.array(timestamps)

    return np.array(list(zip(latitudes, longitudes, timestamps)))

### GPS point interpolation


In [197]:
# Calculate the distance between two points on the Earth's surface using Haversine formula.
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2) * math.sin(d_lat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2) * math.sin(
        d_lon / 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d

In [198]:
# used to extend line segments for finer control.
def intermediate_points(lat1, lon1, lat2, lon2, n):
    intermediate_points = []
    total_distance = haversine_distance(lat1, lon1, lat2, lon2)
    segment_distance = total_distance / (n + 1)  # n+2 points including the start and end points
    for i in range(1, n + 1):
        f = segment_distance * i / total_distance
        lat = lat1 + f * (lat2 - lat1)
        lon = lon1 + f * (lon2 - lon1)
        intermediate_points.append((lat, lon))
    return intermediate_points

In [199]:
# Find the closest pair of points between two sets of points.
def closest_pair(points1, points2):
    min_distance = float("inf")
    closest_point_pair = None
    for p1 in points1:
        for p2 in points2:
            d = haversine_distance(p1[0], p1[1], p2[0], p2[1])
            if d < min_distance:
                min_distance = d
                closest_point_pair = (p1, p2)
    return closest_point_pair, min_distance

### Step 1: temporal - find passengers who travels at this time

1. We query the driver document
2. Use the departure to do a temporal range query on passengers.

In [200]:
try:
    document_id = uuid.UUID("71a2aa23-f60f-44ba-847c-10ce2928fecc")
    driver = database["rides"].find_one({"_id": document_id})

    if driver:
        print(f"document found for: {document_id}")
    else:
        print(f"no document found for: {document_id}")
except Exception as e:
    print(f"failed with {e}")

document found for: 71a2aa23-f60f-44ba-847c-10ce2928fecc


In [201]:
from pydantic import BaseModel, UUID4, Field
from datetime import datetime

class MyModel(BaseModel):
    id: UUID4 = Field(default_factory=uuid.uuid4)
    timestamp: datetime = Field(default_factory=datetime.now)

# Example document
document = {
    "id": uuid.uuid4(),
    "timestamp": datetime.now()
}

# Serialize document
model = MyModel(**document)
serialized = model.model_dump_json()
print(serialized)

# Deserialize document
deserialized = MyModel.model_validate_json(serialized)
print(deserialized.model_dump())


{"id":"f67e3ce1-963d-477a-8aac-e9760f4bbbef","timestamp":"2024-05-15T02:28:56.644678"}
{'id': UUID('f67e3ce1-963d-477a-8aac-e9760f4bbbef'), 'timestamp': datetime.datetime(2024, 5, 15, 2, 28, 56, 644678)}


In [202]:
# make json serializeable
driver["_id"] = str(driver["_id"])
driver["driver"] = str(driver["driver"])

model = Ride(**driver) 
serialized = model.model_dump_json()
await rdb.json().set("driver", ".", serialized)

True

In [203]:
serialized

'{"id":"71a2aa23-f60f-44ba-847c-10ce2928fecc","driver":"b98656dd-42c4-4bd7-978f-a00173c0e9fb","seats_total":3,"seats_available":3,"start_address":{"street":"Dragør","country":"Danmark","city":"Dragør","province":"Region Hovedstaden","postal_code":"","coordinate":{"latitude":55.5942392,"longitude":12.6701646}},"destination_address":{"street":"Herning","country":"Danmark","city":"Herning","province":"Region Midtjylland","postal_code":"","coordinate":{"latitude":56.1375702,"longitude":8.9750399}},"passengers":[],"departure":"2024-05-15T08:00:52Z","max_deviation":5000,"status":"open","stops":[],"route":{"distance":312579.156,"expected_travel_time":12552.647,"shape":{"type":"LineString","coordinates":[[12.67016,55.5942],[12.631369999999995,55.596320000000055],[12.626519999999998,55.597570000000054],[12.618009999999998,55.60544000000006],[12.61264,55.61256000000006],[12.605659999999999,55.61904000000006],[12.60377,55.62848000000007],[12.60283,55.62964],[12.582020000000002,55.62798999999999],

In [204]:
# Deserialize document
deserialized = Ride.model_validate_json(serialized)

In [205]:
deserialized

Ride(id=UUID('4a9efe65-1b92-4b8c-bf3e-8c65546aee6e'), driver=UUID('b98656dd-42c4-4bd7-978f-a00173c0e9fb'), seats_total=3, seats_available=3, start_address=Address(street='Dragør', country='Danmark', city='Dragør', province='Region Hovedstaden', postal_code='', coordinate=Coordinate(latitude=55.5942392, longitude=12.6701646)), destination_address=Address(street='Herning', country='Danmark', city='Herning', province='Region Midtjylland', postal_code='', coordinate=Coordinate(latitude=56.1375702, longitude=8.9750399)), passengers=[], departure=datetime.datetime(2024, 5, 15, 8, 0, 52, tzinfo=TzInfo(UTC)), max_deviation=5000, status=<RideStatus.open: 'open'>, stops=[], route=Route(distance=312579.156, expected_travel_time=12552.647, shape=LineString(type='LineString', coordinates=[[12.67016, 55.5942], [12.631369999999995, 55.596320000000055], [12.626519999999998, 55.597570000000054], [12.618009999999998, 55.60544000000006], [12.61264, 55.61256000000006], [12.605659999999999, 55.619040000000

In [206]:
data = await rdb.json().get("driver")

In [207]:
Ride.parse_raw(data)

Ride(id=UUID('bbba9639-4f4b-4b2a-bbd7-6f7a6346ff33'), driver=UUID('b98656dd-42c4-4bd7-978f-a00173c0e9fb'), seats_total=3, seats_available=3, start_address=Address(street='Dragør', country='Danmark', city='Dragør', province='Region Hovedstaden', postal_code='', coordinate=Coordinate(latitude=55.5942392, longitude=12.6701646)), destination_address=Address(street='Herning', country='Danmark', city='Herning', province='Region Midtjylland', postal_code='', coordinate=Coordinate(latitude=56.1375702, longitude=8.9750399)), passengers=[], departure=datetime.datetime(2024, 5, 15, 8, 0, 52, tzinfo=TzInfo(UTC)), max_deviation=5000, status=<RideStatus.open: 'open'>, stops=[], route=Route(distance=312579.156, expected_travel_time=12552.647, shape=LineString(type='LineString', coordinates=[[12.67016, 55.5942], [12.631369999999995, 55.596320000000055], [12.626519999999998, 55.597570000000054], [12.618009999999998, 55.60544000000006], [12.61264, 55.61256000000006], [12.605659999999999, 55.619040000000

In [208]:
# Define the range for the query (e.g., 2 hours before and after the datetime)
delta_time = 180

delta_prev = driver["departure"] - timedelta(minutes=delta_time)
delta_next = driver["departure"] + timedelta(minutes=delta_time)

# Construct the query to find rides within the specified range
query = {"departure": {"$gte": delta_prev, "$lte": delta_next}}

In [209]:
# Perform the query
results = database["ride_searches"].find(query)

In [210]:
passengers = []

for doc in results:
    passengers.append(doc)

print(passengers)

[{'_id': UUID('edb85552-15ab-4001-8919-ee4eb1d8d7cd'), 'passenger': UUID('941c2cec-cc83-4759-9e6c-6231f40f5ea9'), 'start_address': {'street': 'Roskilde', 'country': 'Danmark', 'city': 'Roskilde', 'province': 'Region Sjælland', 'postal_code': '', 'coordinate': {'latitude': 55.642411, 'longitude': 12.0831694}}, 'destination_address': {'street': 'Slagelse', 'country': 'Danmark', 'city': 'Slagelse', 'province': 'Region Sjælland', 'postal_code': '', 'coordinate': {'latitude': 55.4078186, 'longitude': 11.3570129}}, 'departure': datetime.datetime(2024, 5, 15, 8, 0, 47, tzinfo=<bson.tz_util.FixedOffset object at 0x15fbde1b0>), 'max_deviation': 5000, 'route': {'distance': 59124.738, 'expected_travel_time': 3694.376, 'shape': {'type': 'LineString', 'coordinates': [[12.08316, 55.64241], [12.08166, 55.64171], [12.08131, 55.63999], [12.07605, 55.64013], [12.075310000000002, 55.635439999999974], [12.071680000000002, 55.633389999999984], [12.069480000000004, 55.63097999999999], [11.944190000000004, 5

In [211]:
len(passengers)

7

### Step 2: spatial - Compute points in range (GeoQueries)

In [212]:
max_deviation = 5

In [213]:
in_range_start = []
in_range_dest = []

In [214]:
# iterate over driver geo points for start candidates
for d_point in driver["route"]["shape"]["coordinates"]:
    # check each passengers starting point
    for p in passengers:
        p_points = p["route"]["shape"]["coordinates"]
        dist = haversine_distance(d_point[0], d_point[1], p_points[0][0], p_points[0][1])
        if dist <= max_deviation:
            if p not in in_range_start:
                in_range_start.append(p)

In [215]:
# Pre-calculate distances between driver's route points and start/end points of each passenger
# passenger_distances = {}
# for i, p in enumerate(passengers):
#     p_start = p["route"]["shape"]["coordinates"][0]
#     p_end = p["route"]["shape"]["coordinates"][-1]

#     start_distances = [haversine_distance(d_point[0], d_point[1], p_start[0], p_start[1]) for d_point in driver["route"]["shape"]["coordinates"]]
#     end_distances = [haversine_distance(d_point[0], d_point[1], p_end[0], p_end[1]) for d_point in driver["route"]["shape"]["coordinates"]]
#     passenger_distances[i] = {"start": start_distances, "end": end_distances}

In [216]:
# iterate over driver geo points for destination candidates
for d_point in driver["route"]["shape"]["coordinates"]:
    # check passengers that passed start check
    for p in in_range_start:
        p_points = p["route"]["shape"]["coordinates"]
        dist = haversine_distance(d_point[0], d_point[1], p_points[-1][0], p_points[-1][1])
        if dist <= max_deviation:
            if p not in in_range_dest:
                in_range_dest.append(p)

In [217]:
# Iterate over driver geo points
# for idx, d_point in enumerate(driver["route"]["shape"]["coordinates"]):
#     # Check start and end points for each passenger
#     for i, dists in passenger_distances.items():
#         start_dist = dists["start"][idx]
#         end_dist = dists["end"][idx]
#         if start_dist <= driver["max_deviation"]:
#             in_range_start.append(i)
#         if end_dist <= driver["max_deviation"] and i in in_range_start:
#             in_range_dest.append(i)

In [218]:
len(in_range_start)

4

In [219]:
len(in_range_dest)

3

In [220]:
candidates = [p for p in in_range_start if p in in_range_dest]

In [221]:
for candidate in candidates:
    print(f"rute: ({candidate['start_address']['street']}, {candidate['destination_address']['street']})")

rute: (A C Meyers Vænge 15, Hundige Stationsvej 2)
rute: (Fredericia, Vejle)
rute: (Vejle, Herning)


### Visualize Candidates

In [222]:
fig = go.Figure()

# add passenger and driver trajectories as traces
for candidate in candidates:
    df_c = geojson_to_df(candidate)
    fig.add_trace(go.Scattermapbox(mode="lines", lon=df_c["lon"], lat=df_c["lat"], line_color="green"))

    
# fig.add_trace(go.Scattermapbox(mode="lines", lon=df_p["lon"], lat=df_p["lat"], line_color="blue"))
fig.add_trace(go.Scattermapbox(mode="lines", lon=df_d["lon"], lat=df_d["lat"], line_color="purple"))

fig.update_layout(height=600, mapbox=dict(style="open-street-map", zoom=6, center=dict(lon=df_p["lon"].mean(), lat=df_p["lat"].mean())))
fig.update_layout(dragmode=False, uirevision="lock")

### Deprecated GeoSearch Code

In [223]:
max_distance = driver["max_deviation"]
route = driver["route"]["shape"]["coordinates"]

In [224]:
# TODO: Not working with data-structures: 
# Query documents within the specified distance from the reference point
point_queries = []
for point in route:
    query = {"shape": {"$nearSphere": {"$geometry": point, "$maxDistance": max_distance}}}
    point_queries.append(query)

### Step 3: correlation - Finding optimal matches from candidate list

In [225]:
candidate_queue = heapq.heapify([])

In [226]:
df_d = geojson_to_df(driver)

short_list = []
for candidate in candidates:
    df_c = geojson_to_df(candidate)
    match_tuple = (dtw(df_d, df_c), candidate["passenger"], candidate["_id"], driver["_id"])
    short_list.append(match_tuple)

In [227]:
# Create a heapify distances list
heapq.heapify(short_list)

# To get the pair with the lowest distance, you can simply pop from the heap
for seat in range(driver["seats_available"]):
    distance, pid, sid, did = heapq.heappop(short_list)
    print(distance)

23.823781402346736
25.824822616610593
33.818128265804354


In [228]:
async def create_match(match_tuple):
    # Convert partial response body to dict
    dict = {}

    # Add metadata
    dict["created_at"] = datetime.now()

    # populate match
    dict["similarity"]   = match_tuple[0]
    dict["passenger_id"] = match_tuple[1]
    dict["search_id"]    = match_tuple[2]
    dict["ride_id"]      = match_tuple[3]
    dict["seen"]         = False

    # Validate and create a Report instance
    populated = SaveRequest(**dict)

    # Insert report into the database
    document = populated.model_dump(by_alias=True, exclude=["id"])
    result = db.matches.insert_one(document)

    if not result:
        logger.warning("was not able to create match result")

### Close Database Connections after run

In [229]:
client.close()

In [230]:
await rdb.close()